In [1]:
import sys
sys.path.insert(0,'/home/djl34/kl_git/scripts')
from notebook_imports import *

/home/djl34/mambaforge/envs/raklette/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [2]:
client = Client()

In [3]:
filename = os.path.join(KL_data_dir, "results/method_validation/footprints/zoonomia/{chrom}.tsv")

ddf = dd.read_csv([filename.format(chrom = chrom) for chrom in all_chrom_set], sep = "\t", dtype={'Spliceai_info': 'object', 'DHS_tissue': 'object', 'footprint_motif_clusters': 'object'},
                  usecols=['mu', 'AC_total', 'denovo_an_case', 'denovo_an_control', 'denovo_halldorsson_control'])
ddf = ddf.fillna(0)
# df = ddf.compute()

In [6]:
ddf

,mu,AC_total,denovo_an_case,denovo_an_control,denovo_halldorsson_control,denovo
npartitions=627,,,,,,
,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [4]:
ddf["denovo"] = ddf["denovo_an_case"] + ddf["denovo_an_control"] + ddf["denovo_halldorsson_control"]

In [5]:
ddf.head()

/home/djl34/mambaforge/envs/raklette/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


,mu,AC_total,denovo_an_case,denovo_an_control,denovo_halldorsson_control,denovo
0,0.030,0.0,0.0,0.0,0.0,0.0
1,0.013,0.0,0.0,0.0,0.0,0.0
2,0.073,1.0,0.0,0.0,0.0,0.0
3,0.073,0.0,0.0,0.0,0.0,0.0
4,0.094,0.0,0.0,0.0,0.0,0.0


In [7]:
# try to see if I can reduce partition number to make computation faster
ddf = ddf[["mu", "AC_total", "denovo"]].repartition(partition_size="300MB")

In [5]:
ddf["polymorphic"] = ddf["AC_total"] > 0

In [18]:
columns = ["mu", "denovo_an_case", "denovo_an_control", "denovo_halldorsson_control", "denovo"]

In [11]:
ddf["denovo"] = ddf["denovo_an_case"] + ddf["denovo_an_control"] + ddf["denovo_halldorsson_control"]

In [6]:
len(ddf)

176923101

# 1. First try grouping all of the sites together

In [41]:
polymorphic_sum

mu                            6283649.441
denovo_an_case                   1578.000
denovo_an_control                1526.000
denovo_halldorsson_control       2212.000
dtype: float64

In [11]:
polymorphic_sum = ddf[ddf["polymorphic"]][columns].sum().compute()
monomorphic_sum = ddf[~ddf["polymorphic"]][columns].sum().compute()

In [37]:
sum = pd.DataFrame(polymorphic_sum/polymorphic_sum["mu"])
sum.rename({0: "polymorphic"}, axis = 1, inplace = True)

In [13]:
sum

<function sum(iterable, /, start=0)>

In [38]:
sum["monomorphic"] = monomorphic_sum/monomorphic_sum["mu"]

In [39]:
sum.reset_index(inplace = True)

In [42]:
sum["polymorphic"]/sum["monomorphic"]

0    1.000000
1    1.266794
2    1.271278
3    1.224662
dtype: float64

Ratio of observed SNV / non SNV sites for *de novo* mutations seem to be around 1.25

# 2. Let's try grouping by mutation rate

In [9]:
polymorphic_sum = ddf[ddf["polymorphic"]].groupby("mu")[columns].sum().compute()

In [15]:
sum = pd.DataFrame(polymorphic_sum)

In [21]:
sum["number_of_sites_SNV"] = ddf[ddf["polymorphic"]].groupby("mu").size().compute()

In [17]:
sum["denovo"] = sum["denovo_an_case"] + sum["denovo_an_control"] + sum["denovo_halldorsson_control"]

In [19]:
monomorphic_sum = ddf[~ddf["polymorphic"]].groupby("mu")[columns].sum().compute()

In [26]:
sum["number_of_mutations_no_SNV"] = monomorphic_sum["denovo"]
sum["number_of_sites_no_SNV"] = ddf[~ddf["polymorphic"]].groupby("mu").size().compute()

In [22]:
sum

,mu,denovo_an_case,denovo_an_control,denovo_halldorsson_control,denovo,number_of_sites_SNV
mu,,,,,,
0.013,4321.434,0.0,1.0,0.0,1.0,332418
0.020,34539.040,9.0,8.0,13.0,30.0,1726952
0.030,61631.760,17.0,19.0,27.0,63.0,2054392
0.041,76907.185,18.0,16.0,29.0,63.0,1875785
0.051,87340.611,17.0,25.0,27.0,69.0,1712561
0.062,101563.998,17.0,24.0,37.0,78.0,1638129
0.073,113941.320,28.0,24.0,44.0,96.0,1560840
0.083,124416.087,19.0,34.0,45.0,98.0,1498989
0.094,139717.182,28.0,39.0,55.0,122.0,1486353


In [25]:
monomorphic_sum

,mu,denovo_an_case,denovo_an_control,denovo_halldorsson_control,denovo
mu,,,,,
0.013,123328.231,24.0,23.0,32.0,79.0
0.020,625953.140,139.0,133.0,174.0,446.0
0.030,756996.900,166.0,145.0,164.0,475.0
0.041,694515.359,139.0,141.0,222.0,502.0
0.051,614716.566,130.0,126.0,191.0,447.0
0.062,579097.360,123.0,99.0,189.0,411.0
0.073,545748.219,113.0,108.0,156.0,377.0
0.083,512199.640,110.0,79.0,149.0,338.0
0.094,501415.176,89.0,114.0,139.0,342.0
